In [ ]:
import pandas as pd
import os
import psycopg2

In [ ]:
work_path = r"D:\DataWork\AllWerfenChinaTop\posgres_source\topsta_generalLog\Data"
path_testvol = work_path + "\\CountersForAllTest\\"
path_ostask = work_path + "\\osTask\\"
path_swversion= work_path + "\\SoftwareVersions\\"

In [ ]:
conn = psycopg2.connect(dbname= "mydb", user= "sirius", password= "biicf", host= "localhost")
cur = conn.cursor()

In [ ]:
# TestVol
current_path = path_testvol
files = os.listdir(current_path)
for f in files:
    basename = f.split(".")[0]
    f_abspath = current_path  + f
    print(f_abspath)
    df = pd.read_csv(f_abspath, sep="\t", encoding="utf_16_le")
    if df.empty:
        continue
    df["topsn"], df["date"] = basename.split("_")
    for id, rw in df.iterrows():
        cur.execute("INSERT INTO service.topsta_testvl \
                    (topsn, date, test_name, volume) \
                    VALUES (%s, %s, %s, %s)",
                    ( rw["topsn"], rw["date"], rw["testCode"], rw["total"] ) 
        )

In [ ]:
# os task
current_path = path_ostask
files = os.listdir(current_path)
for f in files:
    basename = f.split(".")[0]
    f_abspath = current_path + f
    print(f_abspath)
    try:
        with open(f_abspath, "r") as f:
            first_line = f.readline().strip()
            skip_num = 0 if first_line else 1
        df = pd.read_csv(f_abspath, sep=",", encoding="utf_8", skiprows= skip_num)
    except:
        continue
    if df.empty:
        continue
    cur.execute("DELETE FROM service.topsta_ostask WHERE topsn = %s", (basename.split("_")[0],))
    df["topsn"], df["date"] = basename.split("_")
    for id, rw in df.iterrows():
        cur.execute("INSERT INTO service.topsta_ostask \
                    (topsn, date, program, pid, session_name, session_num, memory) \
                    VALUES (%s, %s, %s, %s, %s, %s, %s)",
                    ( rw["topsn"], rw["date"], rw["Image Name"], rw["PID"], rw["Session Name"], rw["Session#"], rw["Mem Usage"] ) 
        )

In [ ]:
# software version
current_path = path_swversion
files = os.listdir(current_path)
for f in files:
    basename = f.split(".")[0]
    f_abspath = current_path + f
    print(f_abspath)
    df = pd.read_csv(f_abspath, sep="\t", encoding="utf_16_le", skiprows= 0)
    if df.empty:
        continue
    cur.execute("DELETE FROM service.topsta_swvsn WHERE topsn = %s", (basename.split("_")[0],))
    df["topsn"], df["date"] = basename.split("_")
    for id, rw in df.iterrows():
        cur.execute("INSERT INTO service.topsta_swvsn \
                    (topsn, date, index, main_version, creation, description, install_value, expected_value) \
                    VALUES (%s, %s, %s, %s, %s, %s, %s, %s)",
                    ( rw["topsn"], rw["date"], rw["id"], rw["mainSystemVersion"], rw["creation"], rw["description"], rw["installedValue"], rw["expectedValue"] ) 
        )

In [ ]:
conn.commit()

In [ ]:
cur.close()
conn.close()